In [1]:
from search_libraries import *
import time

In [2]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.core.display import HTML

In [3]:
from configparser import ConfigParser
import requests
from bs4 import BeautifulSoup

config = ConfigParser()
config.read('.env')

['.env']

In [4]:
def search_goodreads(search_term):
    key = config.get('goodreads','key')
    url = 'https://www.goodreads.com/search/index.xml'
    payload = {'q': search_term, 'key': key }
    r = requests.get(url, params=payload)
    soup = BeautifulSoup(r.content, 'html.parser')
    titles = ['Please select a title...']
    for title in soup.find_all('title'):
        titles.append(title.string)
    return widgets.Select(
        options=titles,
        value=titles[0],
        description='Results:',
        disabled=False
    )

In [5]:
def style_result(result):
    return '<div>' + result.title + ' <span class="badge badge-secondary"> '+result.library.title() +' </span></div>'

def style_results(results):
    html = ''
    for result in results:
        html += style_result(result)
    return HTML(html)

In [6]:
def on_value_change(change):
    search_term = change['new']
    if '(' in search_term: # series books screw up library searches
        search_term, _ = search_term.split('(')
    print('Loading library search results for: ', search_term, '...')
    results = search_libraries(search_term)
    if len(results) == 0:
        print('No results found')
    else:
        display(style_results(results))

In [7]:
@interact_manual(search="")
def search(search):
    """
    Widget for library search.
    
    Search Yarra, Melbourne and Moreland library catalogues with the search term entered in the search box.
    
    Args:
        search (str): The search term
    Returns:
        list: The list of zipped (as in zip()) results for catalogue searches.
    """
    titles = []
    if len(search) > 3:
        selector = search_goodreads(search)
        selector.observe(on_value_change, names='value')
        display(selector)
        time.sleep(1) # sleep 1 second to ensure not exceeding API terms
    else:
        print("Need at least 3 characters to search")

interactive(children=(Text(value='', description='search'), Button(description='Run Interact', style=ButtonStyle()), Output()), _dom_classes=('widget-interact',))